In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, normalize
import keras
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Dense, Dropout
from keras.metrics import CategoricalAccuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Cleaned_Dataset.csv')

In [3]:
del df['Unnamed: 0']

In [4]:
df.shape

(1217007, 21)

In [5]:
df.head()

,Source Port,Protocol,Flow Duration,Fwd Packet Length Min,Bwd Packet Length Min,Fwd IAT Total,Bwd IAT Total,Fwd PSH Flags,Fwd Packets/s,Min Packet Length,...,ACK Flag Count,URG Flag Count,CWE Flag Count,Init_Win_bytes_forward,min_seg_size_forward,Idle Mean,Idle Max,Idle Min,Inbound,Label
0,60675,6,5220876,0.0,0.0,215957.0,5220754.0,0,2.298465,0.0,...,1,0,0,65535,32,0.0,0.0,0.0,1,DrDoS_NTP
1,60676,6,12644252,0.0,0.0,12644252.0,1.0,0,0.395437,0.0,...,1,0,0,65535,32,0.0,0.0,0.0,1,DrDoS_NTP
2,443,6,3,0.0,0.0,3.0,0.0,0,666666.666667,0.0,...,0,1,0,0,20,0.0,0.0,0.0,1,BENIGN
3,443,6,3,0.0,0.0,3.0,0.0,0,666666.666667,0.0,...,0,1,0,0,20,0.0,0.0,0.0,1,BENIGN
4,0,0,114329232,0.0,0.0,114329232.0,0.0,0,0.454827,0.0,...,0,0,0,-1,0,9527428.0,9950741.0,9092248.0,0,BENIGN


In [6]:
df[' Label'].value_counts()

DrDoS_NTP    1202642
BENIGN         14365
Name:  Label, dtype: int64

In [8]:
X = normalize( ( df.loc[:, df.columns != ' Label'] ).values )
y = df[' Label']
label_encoder = LabelEncoder()
y = y.map(lambda value : 0 if value == 'BENIGN' else 1)
y = keras.utils.to_categorical(y, num_classes=y.nunique())

In [10]:
random_state = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, stratify=y, random_state=random_state)

In [12]:
accuracy = CategoricalAccuracy() 
tp = TruePositives()
tn = TrueNegatives()
fp = FalsePositives()
fn = FalseNegatives()
metrics = [accuracy, tp, tn, fp, fn]

model = Sequential()
model.add(Dense(64, input_shape=(len(df.columns) - 1 ,))) 
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(8))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

model.fit(x=X_train, y=y_train, epochs=20, batch_size=512, verbose=0, validation_data=(X_test, y_test))

scores = model.evaluate(X_test, y_test, verbose=2)
acc, loss, tpn, tnn, fpn, fnn = scores[1]*100, scores[0]*100, scores[2], scores[3], scores[4], scores[5]
totaln = tpn + tnn + fpn + fnn
print('Baseline: accuracy: {:.2f}%: loss: {:2f}'.format(acc, loss))
print('\tTrue Positive Rate: {} ({})'.format(tpn/totaln, tpn))
print('\tTrue Negative Rate: {} ({})'.format(tnn/totaln, tnn))
print('\tFalse Positive Rate: {} ({})'.format(fpn/totaln, fpn))
print('\tFalse Negative Rate: {} ({})'.format(fnn/totaln, fnn))

Baseline: accuracy: 99.75%: loss: 0.727657
	True Positive Rate: 0.49874506054335893 (400605.0)
	True Negative Rate: 0.49874506054335893 (400605.0)
	False Positive Rate: 0.0012549394566410948 (1008.0)
	False Negative Rate: 0.0012549394566410948 (1008.0)
